# Generate object(sign) and annotations on draw

In [104]:
%matplotlib inline

import os
from glob import glob
from matplotlib import pyplot
from PIL import Image
import numpy as np

from lxml import etree
import xml.etree.ElementTree as xml
import random
import cv2

In [105]:
draw_data = glob('./data/backGround/draw/*.png')
annotaions_data = glob('./data/backGround/annotations/*.xml')
sign_data = glob('./data/backGround/sign/*.png')

# Reshape Sign 
size = 20, 20

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im.save(s)

# Change color of sign

In [83]:
def ChangeColor(inputImg,r2,g2,b2):
    
    im = cv2.imread(inputImg)
    data = np.array(im)
    
    imName = inputImg.split('sign/')[1]
    imName = imName.split('.png')[0]    

    r1, g1, b1 = 0, 0, 0 # Original value
    r2, g2, b2 = r2,g2,b2 # Value that we want to replace it with

    red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
    mask = (red == r1) & (green == g1) & (blue == b1)
    data[:,:,:3][mask] = [r2, g2, b2]

    im = Image.fromarray(data)    
    imDir = './data/backGround/sign/COR_'+imName+'.png'
    im.save(imDir)
    
    return imDir

# Make n-Sign on Draw with loaction(x,y) information 

In [122]:

def genData(clss, amount, dr_fileName,r2,g2,b2):
    
    l_img =  cv2.imread(dr_fileName)
    s_img =  cv2.imread('./data/backGround/sign/'+clss+'.png',-1)  
    
    signDir = './data/backGround/sign/'+clss+'.png'
    
    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    
    dr_fileName = dr_fileName.split('draw/')[1]
    dr_fileName = dr_fileName.split('.png')[0]    
    
    new_dirFileName = './data/draw/dataset/GN_'+dr_fileName+str(random.random())+'.png'          
    new_fileName  = new_dirFileName .split('dataset/')[1]

    imName = ChangeColor(signDir,r2,g2,b2)
    s_img =  cv2.imread(imName,-1)
    
    h,w,d = l_img.shape
   
    
    # make annotations file
    root = etree.Element("annotation")

    folder = etree.SubElement(root, "folder").text ='draw'  
    filename =  etree.SubElement(root, "filename").text = new_fileName    
    path =  etree.SubElement(root, "path").text = new_dirFileName

    source =  etree.SubElement(root, "source")
    database =  etree.SubElement(source, "database").text= 'Unknown'     
    
    size =  etree.SubElement(root, "size")
    width =  etree.SubElement(size, "width").text = str(w)
    height =  etree.SubElement(size, "height").text = str(h)
    depth =  etree.SubElement(size, "depth").text = str(d)

    segmented =  etree.SubElement(root, "segmented").text = '0'
    
       
    # generate signs and annotaion files.                 
    for i in range(amount):    
               
        xmin = random.randrange(100,w)
        ymin = random.randrange(100,h)
                
        #x_offset=y_offset=16
        
        x1, x2 = xmin,  xmin + s_img.shape[1] 
        y1, y2 = ymin,  ymin + s_img.shape[0] 
        
        print(x1, x2, y1, y2)
        
        obj =  etree.SubElement(root, "object")
        name =   etree.SubElement(obj, "name").text = clss
        poset =   etree.SubElement(obj, "pose").text = 'Unspecified'
        truncated =   etree.SubElement(obj, "truncated").text = '0'
        difficult =   etree.SubElement(obj, "difficult").text = '0'
        
        bndbox = etree.SubElement(obj, "bndbox")
        xmin =  etree.SubElement(bndbox, "xmin").text = str(x1)
        ymin =  etree.SubElement(bndbox, "ymin").text = str(y1)
        xmax =  etree.SubElement(bndbox, "xmax").text = str(x2)
        ymax =  etree.SubElement(bndbox, "ymax").text = str(y2)
        
        for c in range(0, 3):
            l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])

    cv2.imwrite(new_fileName, l_img)        
    tree = etree.ElementTree(root)
    
    with open('./data/backGround/'+new_fileName+'.xml', "bw") as fh:
        tree.write(fh)
         
       

In [123]:
genData('DT_150',10, './data/backGround/draw/b1.png',255,228,0)

1185 1205 930 941
1324 1344 727 738
952 972 739 750
1224 1244 139 150
1426 1446 781 792
329 349 228 239
460 480 782 793
1429 1449 322 333
1431 1451 161 172
614 634 935 946
